In [46]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from sklearn.model_selection import train_test_split

# Import and clean datas

In [47]:
train_df = pd.read_csv("../datas/train_data.csv")

In [48]:
def remove_newlines(df):
    df = df.replace("\n", '', regex=True)
    return df
train_df = remove_newlines(train_df)

In [49]:
train_df.head()

,ID,filename,texte,sexe,date_accident,date_consolidation
0,0,Agen_100515.txt,Le : 12/11/2019 Cour d’appel d’Agen chambre ...,homme,1991-04-09,n.c.
1,1,Agen_1100752.txt,Le : 12/11/2019 Cour d’appel d’Agen chambre ...,homme,2005-06-10,2010-01-19
2,2,Agen_1613.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience...,femme,1997-09-26,n.c.
3,3,Agen_2118.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience...,femme,1982-08-07,1982-11-07
4,4,Agen_21229.txt,Le : 12/11/2019 Cour d’appel d’Agen Audience...,homme,1996-11-26,n.c.


# Only keep the sex column

In [50]:
train = train_df.drop(["date_accident", "date_consolidation"], axis=1)

In [51]:
nltk.download('stopwords')
stop_words = stopwords.words('french')
train['texte'] = train["texte"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/SamuelLP/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
train = train.drop(["ID", "filename"], axis=1)

In [53]:
train['text_id'] = np.arange(len(train))

# Let's predict the sex of the victim!

## We will use a TF-IDF

In [62]:
X = train[["texte", 'text_id']]

vect = TfidfVectorizer(
  max_features=5000,
  min_df=20,
  stop_words=list(fr_stop),
  ngram_range=(2, 2),
  binary=True)

X = vect.fit_transform(train['texte'])
train["sexe"] = train["sexe"].replace({'homme':0, "femme":1, "n.c.": -1})
y = train['sexe']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = LogisticRegression(random_state=42).fit(X, y)

preds = clf.predict((X_test))

/Users/SamuelLP/Desktop/git/projet_nlp/.nlp_venv/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['neuf', 'qu', 'quelqu'] not in stop_words.
  warnings.warn(


# Now, we compute our metrics

In [63]:
print("accuracy:", round(accuracy_score(y_test, preds), 2) * 100, "%")
print("f1:", round(f1_score(y_test, preds, average='macro'), 2) * 100, "%")

accuracy: 92.0 %
f1: 89.0 %
